**Workflow**:
1. Get the data sheet and checklist sheet (in xlsx format) that I've uploaded on Google Drive. Since I'm working on Google Colab, mount my drive's contents.
2. Get the stocks name and their respective number of rows which correspond to number of rows. (Don't rely on max_row since empty cells are counted too)
3. Extact the table of 22 stocks into 22 seperated file.
4. Convert all files into csv files.
5. Zip all files

**Challenges**:
1. Huge amount of data. Inefficient to store on own device or Google drive.
2. Impractical to share all the file manually (cloud storage needed)

In [ ]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import openpyxl
from openpyxl import Workbook

wb = openpyxl.load_workbook("/content/drive/MyDrive/Capital Dynamics - Dataset.xlsx")
sheet_data = wb.get_sheet_by_name("Data Set")
sheet_checklist = wb.get_sheet_by_name("Checklist")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  


In [ ]:
# total number of rows
print("# Rows:",sheet_data.max_row)

# total number of columns
print("# Columns:", sheet_data.max_column)

# Rows: 9100
# Columns: 153


In [ ]:
# all columns headers (each of them seperated by 7 columns)
stocks_name = []
for i in range (1, sheet_data.max_column +1, 7):
    cell = sheet_data.cell(row = 1, column = i)
    if (cell.value):  # in case we include empty cell
      stocks_name.append(cell.value)  # value at cell(1, i)

print('Stocks: ', stocks_name)

Stocks:  ['AHEALTH', 'APM', 'BIOHLDG', 'BSTEAD', 'CAPITALA', 'EUPE', 'HPMT', 'ICAP', 'KGB', 'KRONO', 'LUXCHEM', 'MKH', 'OCK', 'OCNCASH', 'PADINI', 'PARKSON', 'SALUTE', 'SAM', 'SURIA', 'TONGHER', 'UTDPLT', 'WELLCAL']


In [ ]:
# extract number of rows for each stock from Checklist sheet
stocks_days = {}

for r in range(4, 26):  # we know the contents of sheet :)
  if (sheet_checklist.cell(row=r, column = 2)): # make sure it's not empty cell
    stocks_days[sheet_checklist.cell(row=r, column = 2).value] = sheet_checklist.cell(row=r, column = 4).value

for stock, days in stocks_days.items():
  print(stock,"has",days,"rows.")

AHEALTH has 4475 rows.
APM has 5152 rows.
BIOHLDG has 1746 rows.
BSTEAD has 9095 rows.
CAPITALA has 4293 rows.
EUPE has 5328 rows.
HPMT has 725 rows.
ICAP has 3874 rows.
KGB has 2670 rows.
KRONO has 1826 rows.
LUXCHEM has 3308 rows.
MKH has 6136 rows.
OCK has 2413 rows.
OCNCASH has 3234 rows.
PADINI has 5060 rows.
PARKSON has 3592 rows.
SALUTE has 1475 rows.
SAM has 5705 rows.
SURIA has 6225 rows.
TONGHER has 5293 rows.
UTDPLT has 7889 rows.
WELLCAL has 3761 rows.


In [104]:
# create new Google Drive folder
import os
path = "drive/MyDrive/hackathon_data"
os.mkdir(path)

In [94]:
# formatting datetime
# we'll format the cell when copying the contents
from openpyxl.styles import NamedStyle

date_style = NamedStyle(name="formatted_datetime",
                        number_format = "DD/MM/YYYY HH:MM:MM")

In [100]:
wb1 = openpyxl.load_workbook("/content/drive/MyDrive/Capital Dynamics - Dataset.xlsx")
ws1 = wb1.active

def create_copy_into_new_file(file_name, min_r, max_r, min_c, max_c):
  # create new file
  new_wb = Workbook()
  new_file_name = file_name + ".xlsx"
  new_wb.save("/content/drive/MyDrive/hackathon_data/"+new_file_name)

  # copy contents into new file accordingly to stock
  wb2 = openpyxl.load_workbook("/content/drive/MyDrive/hackathon_data/"+new_file_name)
  wb2.add_named_style(date_style)
  ws2 = wb2.active
  for row in ws1.iter_rows(min_row = min_r, max_row = max_r, min_col= min_c, max_col=max_c ):
    ws2.append((cell.value for cell in row))
  
  # save the modified file
  wb2.save("/content/drive/MyDrive/hackathon_data/"+new_file_name)

# start and end column number for a stock
start_col = 1
end_col = 6

for name, days in stocks_days.items():
  create_copy_into_new_file(name, 2, days+2, start_col, end_col)
  start_col = start_col + 7
  end_col = end_col + 7

In [103]:
# convert to csv files using pandas library
# ####### date in excel file look OK in csv file
import pandas as pd

def excel_to_csv():
  for file in stocks_name:
    file_name = file + ".xlsx"
    read_excel_file = pd.read_excel("/content/drive/MyDrive/hackathon_data/"+file_name)
    new_file_name = file + ".csv"
    read_excel_file.to_csv(("/content/drive/MyDrive/hackathon_data/"+new_file_name), index = None, header = True)
excel_to_csv()

In [117]:
# delete xlsx files using Linux command since Google Colab run on Ubuntu
!ls

# delete all excel files in the specified path
!rm drive/MyDrive/hackathon_data/*xlsx

drive  sample_data


In [124]:
# zip the files and download the zipped file
!zip -r hackathon_data.zip /content/drive/MyDrive/hackathon_data

from google.colab import files
files.download("/content/hackathon_data.zip")

updating: content/drive/MyDrive/hackathon_data/ (stored 0%)
updating: content/drive/MyDrive/hackathon_data/.ipynb_checkpoints/ (stored 0%)
updating: content/drive/MyDrive/hackathon_data/AHEALTH.csv (deflated 79%)
updating: content/drive/MyDrive/hackathon_data/APM.csv (deflated 78%)
updating: content/drive/MyDrive/hackathon_data/BIOHLDG.csv (deflated 79%)
updating: content/drive/MyDrive/hackathon_data/BSTEAD.csv (deflated 77%)
updating: content/drive/MyDrive/hackathon_data/CAPITALA.csv (deflated 74%)
updating: content/drive/MyDrive/hackathon_data/EUPE.csv (deflated 78%)
updating: content/drive/MyDrive/hackathon_data/HPMT.csv (deflated 77%)
updating: content/drive/MyDrive/hackathon_data/ICAP.csv (deflated 81%)
updating: content/drive/MyDrive/hackathon_data/KGB.csv (deflated 77%)
updating: content/drive/MyDrive/hackathon_data/KRONO.csv (deflated 75%)
updating: content/drive/MyDrive/hackathon_data/LUXCHEM.csv (deflated 78%)
updating: content/drive/MyDrive/hackathon_data/MKH.csv (deflated 7

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>